In [18]:
import numpy as np
import pandas as pd
import os
import random
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn import datasets
from sklearn import svm
import seaborn as sns

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [19]:
sns.set()

# Loading the data

In [2]:
file = "../Data/result_full_factorial_pgm.csv"

In [3]:
data = pd.read_csv(file)

In [4]:
data.head()

,Unnamed: 0,Unnamed: 0.1,enzyme_complex_concentration,enzyme_concentration,k1_bwd_effective,k1_bwd_relative,k1_fwd_effective,k1_fwd_relative,k2_bwd_effective,k2_bwd_relative,k2_fwd_effective,k2_fwd_relative,mu_mass,product_concentration,realization,sigma_mass,substrate_concentration,volume_fraction
0,0,0,0.000058,0.000006,4.503135,0.450314,262554.802451,1.723016,7.139138e+05,2.164051,9.907347,0.450334,12.1,0.000098,2.0,0.000,0.000049,0.2
1,1,0,0.000006,0.000058,4.491493,0.449149,285144.598410,1.871261,6.227429e+05,1.887689,9.879449,0.449066,12.1,0.000147,3.0,0.000,0.000245,0.2
2,2,0,0.000019,0.000045,7.636543,0.763654,262351.891789,1.721684,6.597299e+05,1.999806,16.799770,0.763626,31.9,0.000016,5.0,0.825,0.000024,0.2
3,3,0,0.000013,0.000051,0.634308,0.063431,484409.307429,3.178936,1.063666e+06,3.224238,1.395557,0.063434,12.1,0.000196,8.0,0.000,0.000012,0.4
4,4,0,0.000006,0.000058,4.416002,0.441600,281281.994713,1.845913,5.871605e+05,1.779830,9.719614,0.441801,12.1,0.000049,4.0,0.000,0.000147,0.2


In [5]:
data.shape

(151956, 18)

In [6]:
data.dtypes

Unnamed: 0                        int64
Unnamed: 0.1                      int64
enzyme_complex_concentration    float64
enzyme_concentration            float64
k1_bwd_effective                float64
k1_bwd_relative                 float64
k1_fwd_effective                float64
k1_fwd_relative                 float64
k2_bwd_effective                float64
k2_bwd_relative                 float64
k2_fwd_effective                float64
k2_fwd_relative                 float64
mu_mass                         float64
product_concentration           float64
realization                     float64
sigma_mass                      float64
substrate_concentration         float64
volume_fraction                 float64
dtype: object

In [7]:
data.drop('Unnamed: 0.1', axis=1, inplace=True)
data.drop("Unnamed: 0",axis = 1, inplace = True)
data.head()

,enzyme_complex_concentration,enzyme_concentration,k1_bwd_effective,k1_bwd_relative,k1_fwd_effective,k1_fwd_relative,k2_bwd_effective,k2_bwd_relative,k2_fwd_effective,k2_fwd_relative,mu_mass,product_concentration,realization,sigma_mass,substrate_concentration,volume_fraction
0,0.000058,0.000006,4.503135,0.450314,262554.802451,1.723016,7.139138e+05,2.164051,9.907347,0.450334,12.1,0.000098,2.0,0.000,0.000049,0.2
1,0.000006,0.000058,4.491493,0.449149,285144.598410,1.871261,6.227429e+05,1.887689,9.879449,0.449066,12.1,0.000147,3.0,0.000,0.000245,0.2
2,0.000019,0.000045,7.636543,0.763654,262351.891789,1.721684,6.597299e+05,1.999806,16.799770,0.763626,31.9,0.000016,5.0,0.825,0.000024,0.2
3,0.000013,0.000051,0.634308,0.063431,484409.307429,3.178936,1.063666e+06,3.224238,1.395557,0.063434,12.1,0.000196,8.0,0.000,0.000012,0.4
4,0.000006,0.000058,4.416002,0.441600,281281.994713,1.845913,5.871605e+05,1.779830,9.719614,0.441801,12.1,0.000049,4.0,0.000,0.000147,0.2


# From the paper, theory

Linear regression to estimate $\beta$ and $\alpha_j$ in the following model:

\begin{equation*}
log(\frac{k_j}{k_{j,0}}) = \beta_j + \alpha_{E,j}log(\frac{[E]}{[E]_0}) + \alpha_{ES,j}log(\frac{[ES]}{[ES]_0})  + \alpha_{P,j}log(\frac{[P]}{[P]_0})  + \alpha_{S,j}log(\frac{[S]}{[S]_0}) 
\end{equation*}

where $j\in$ [1_bwd, 1_fwd, 2_bwd, 2_fwd],

$k$ is the effective rate constant, and $k_0$ is the relative rate constant

The given concentration is (??) $[X]/[X]_0$

knowing that be modify our data set in order to have the $log(\frac{k_j}{k_{j,0}})$ as a feature we will try to predict

In [8]:
data.dtypes

enzyme_complex_concentration    float64
enzyme_concentration            float64
k1_bwd_effective                float64
k1_bwd_relative                 float64
k1_fwd_effective                float64
k1_fwd_relative                 float64
k2_bwd_effective                float64
k2_bwd_relative                 float64
k2_fwd_effective                float64
k2_fwd_relative                 float64
mu_mass                         float64
product_concentration           float64
realization                     float64
sigma_mass                      float64
substrate_concentration         float64
volume_fraction                 float64
dtype: object

In [9]:
# create transformed dataframe
data_trans = data.copy()
data_trans['y1_bwd'] = np.log(data_trans['k1_bwd_effective']/data_trans['k1_bwd_relative'])
data_trans['y1_fwd'] = np.log(data_trans['k1_fwd_effective']/data_trans['k1_fwd_relative'])
data_trans['y2_bwd'] = np.log(data_trans['k2_bwd_effective']/data_trans['k2_bwd_relative'])
data_trans['y2_fwd'] = np.log(data_trans['k2_fwd_effective']/data_trans['k2_fwd_relative'])
data_trans['E'] = np.log(data_trans['enzyme_concentration'])
data_trans['ES'] = np.log(data_trans['enzyme_complex_concentration'])
data_trans['P'] = np.log(data_trans['product_concentration'])
data_trans['S'] = np.log(data_trans['substrate_concentration'])
data_trans.drop(['k1_bwd_effective','k1_bwd_relative', 'k1_fwd_effective',
                'k1_fwd_relative', 'k2_bwd_effective', 'k2_bwd_relative',
                'k2_fwd_effective', 'k2_fwd_relative', 'enzyme_concentration',
                'enzyme_complex_concentration', 'product_concentration',
                'substrate_concentration'], axis=1, inplace=True)
data_trans.head()

,mu_mass,realization,sigma_mass,volume_fraction,y1_bwd,y1_fwd,y2_bwd,y2_fwd,E,ES,P,S
0,12.1,2.0,0.000,0.2,2.302585,11.934139,12.706535,3.091042,-11.959213,-9.761988,-0.188378,-0.202524
1,12.1,3.0,0.000,0.2,2.302585,11.934139,12.706535,3.091042,-9.761988,-11.959213,-0.180104,-0.169679
2,31.9,5.0,0.825,0.2,2.302585,11.934139,12.706535,3.091042,-10.013302,-10.860600,-0.224945,-0.216670
3,12.1,8.0,0.000,0.4,2.302585,11.934139,12.706535,3.091042,-9.879771,-11.266065,-0.174233,-0.230816
4,12.1,4.0,0.000,0.2,2.302585,11.934139,12.706535,3.091042,-9.761988,-11.959213,-0.202524,-0.180104


In [15]:
data_trans.describe()

,mu_mass,realization,sigma_mass,volume_fraction,y1_bwd,y1_fwd,y2_bwd,y2_fwd,E,ES,P,S
count,1.519560e+05,1.519560e+05,1.519560e+05,1.519560e+05,151956.000000,151956.000000,1.519560e+05,151956.000000,1.519560e+05,1.519560e+05,1.519560e+05,1.519560e+05
mean,4.005054e-13,-4.189661e-16,-1.581754e-13,-4.139837e-14,-16672.173597,123286.060910,-3.726086e-11,-27906.212024,-1.619518e-12,-1.748794e-12,-1.301378e-12,-1.339471e-12
std,1.000003e+00,1.000003e+00,1.000003e+00,1.000003e+00,1.000003,1.000003,0.000000e+00,1.000003,1.000003e+00,1.000003e+00,1.000003e+00,1.000003e+00
min,-1.440188e+00,-1.566638e+00,-6.346179e-01,-1.891130e+00,-16679.668565,123285.859216,-3.726086e-11,-27910.118093,-2.097673e+00,-2.097228e+00,-1.370592e+00,-1.363591e+00
25%,-4.872857e-01,-8.703341e-01,-6.346179e-01,-5.735604e-01,-16672.215630,123285.859216,-3.726086e-11,-27905.705077,-4.775870e-01,-4.773298e-01,-9.354731e-01,-9.289178e-01
50%,6.561966e-01,-1.740302e-01,-6.346179e-01,8.522464e-02,-16672.215630,123285.859216,-3.726086e-11,-27905.705077,2.757101e-01,2.758801e-01,3.250343e-04,5.922250e-03
75%,6.561966e-01,8.704258e-01,1.575751e+00,7.440096e-01,-16672.215630,123285.859216,-3.726086e-11,-27905.705077,7.718942e-01,7.720068e-01,9.361215e-01,9.407605e-01
max,1.174999e+00,1.566730e+00,1.575751e+00,1.402795e+00,-16664.762695,123291.018916,-3.726086e-11,-27903.498569,1.142499e+00,1.142569e+00,1.371242e+00,1.375436e+00


In [10]:
x = data_trans[['E', 'ES', 'P', 'S']].values
X = np.c_[np.ones(len(x)), x]
y = data_trans['y1_bwd']

reg = LinearRegression()
reg.fit(X,y)
print("beta ",reg.intercept_)
print("alpha ", reg.coef_)

beta  2.302585092994046
alpha  [0.00000000e+00 2.86187457e-18 2.65181540e-18 1.96211624e-17
 2.04219902e-17]


From paper we expected to find: -1.48e-02, 1.9e-04, 1.89e-05, -2.94e-03, -2.94e-03

In [11]:
def standardize(x):
    """Standardize a column."""
    mean_x = np.mean(x)
    x = x - mean_x
    std_x = np.std(x)
    if std_x != 0:
        x = x / std_x
    return x, mean_x, std_x

def standardize_data(data):
    '''
    normalization of a data_set
    '''
    for i in range(0,data.shape[1]):
        data.iloc[:,i],mean,variance = standardize(data.iloc[:,i])

    return data

In [12]:
data_test = standardize_data(data_trans)

In [13]:
x = data_test[['E', 'ES', 'P', 'S']].values
X = np.c_[np.ones(len(x)), x]
y = data_test['y1_bwd']

reg = LinearRegression()
reg.fit(X,y)
print("beta ",reg.intercept_)
print("alpha ", reg.coef_)

beta  -16672.173596747387
alpha  [0.         0.01628486 0.01509131 0.00394474 0.00410995]
